In [83]:
import tensorflow as tf 
import os
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

# Keras Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [84]:
data = tf.keras.utils.image_dataset_from_directory("data")

Found 5631 files belonging to 4 classes.


In [85]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [86]:
# Number represents numpy array of images and the labels
scaled = batch[0]/255 

In [87]:
scaled.max()

1.0

# 2. PreProcess Data

In [88]:
data = data.map(lambda x, y: (x/255, y))

In [89]:
scaled_iterator = data.as_numpy_iterator()

In [90]:
scaled_iterator.next()[0].max()

0.9843137

### 2.1 Split Data

In [91]:
len(data)

176

In [92]:
train_size = int(len(data) * .7)
val_size = int(len(data) *.2) + 1
test_size = int(len(data) *.1) + 1

In [93]:
train = data.take(train_size)
test = data.take(test_size)
val = data.take(val_size)

# 3. Deep Model

## 3.1 Build Deep Learning Model

In [94]:
model = Sequential()

In [95]:
model.add(Conv2D(16, (3, 3), 1, activation = "relu", input_shape = (256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3), 1, activation = "relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3), 1, activation = "relu"))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

NameError: name 'Activation' is not defined

In [ ]:
model.compile("adam", loss = tf.losses.BinaryCrossentropy(), metrics = ["accuracy"])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 30, 30, 64)       

## 3.2 Train

In [ ]:
logdir = "logs"
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

In [ ]:
hist = model.fit(train, epochs = 20, validation_data=val, callbacks = [tensorboard_callback])

Epoch 1/20


123/123 [==============================] - 63s 498ms/step - loss: -28590048.0000 - accuracy: 0.2045 - val_loss: -221344272.0000 - val_accuracy: 0.2170
Epoch 2/20
123/123 [==============================] - 59s 477ms/step - loss: -4654143488.0000 - accuracy: 0.2025 - val_loss: -17250410496.0000 - val_accuracy: 0.2231
Epoch 3/20
123/123 [==============================] - 51s 409ms/step - loss: -77446184960.0000 - accuracy: 0.2035 - val_loss: -194690957312.0000 - val_accuracy: 0.2188
Epoch 4/20
123/123 [==============================] - 55s 449ms/step - loss: -491976687616.0000 - accuracy: 0.2010 - val_loss: -977101914112.0000 - val_accuracy: 0.2153
Epoch 5/20
 59/123 [=============>................] - ETA: 26s - loss: -1309663166464.0000 - accuracy: 0.2129

KeyboardInterrupt: 